In [19]:
import pandas as pd

df = pd.read_parquet("data/")

In [20]:
df.loc[4]

brand                                                          Peugeot
model                                                      Partner Van
color                                                            Biały
seats                                                                2
year                                                              2025
fuel                                                            Diesel
capacity                                                     1 499 cm3
power                                                           102 KM
body_type                                             Furgon (blaszak)
gearbox                                                       Manualna
mileage                                                           1 km
condition                                                         Nowy
accident_free                                                      Tak
country_of_origin                                               Polska
title 